In [5]:
pip install selenium webdriver-manager


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import json
import urllib.parse

# Thiết lập tùy chọn logging
options = Options()
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

# Khởi chạy trình duyệt mà không cần tải chromedriver thủ công
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Điều hướng đến trang cần phân tích
driver.get('https://tiki.vn/nha-sach-tiki/c8322')

# Lấy các log từ Network
logs = driver.get_log("performance")

# Danh sách các URL API mà chúng ta quan tâm
api_urls = [
    'https://tiki.vn/api/personalish/v1/blocks/listings',
    'https://tiki.vn/api/v2/products'
]

# Danh sách để lưu trữ các yêu cầu đã tìm thấy
found_requests = []

for log in logs:
    message = json.loads(log['message'])['message']
    # Chỉ xem xét các sự kiện 'Network.requestWillBeSent'
    if message['method'] == 'Network.requestWillBeSent':
        request = message['params']['request']
        url = request['url']
        for api_url in api_urls:
            if url.startswith(api_url):
                # Trích xuất params từ URL
                parsed_url = urllib.parse.urlparse(url)
                params = urllib.parse.parse_qs(parsed_url.query)
                # Kiểm tra nếu 'limit' có giá trị '40'
                if params.get('limit', [''])[0] == '40':
                    # Lấy headers
                    headers = request.get('headers', {})
                    # Thêm vào danh sách
                    found_requests.append({
                        'api_url': api_url,
                        'full_url': url,
                        'params': params,
                        'headers': headers
                    })
                break  # Dừng kiểm tra các API URL khác

# In ra các yêu cầu đã tìm thấy
if found_requests:
    for idx, req in enumerate(found_requests):
        print(f"\nYêu cầu {idx+1}:")
        print(f"API URL: {req['api_url']}")
        print(f"Full URL: {req['full_url']}\n")
        print('Params:')
        for key, value in req['params'].items():
            print(f'{key}: {value}')
        print('\nHeaders:')
        for key, value in req['headers'].items():
            print(f'{key}: {value}')
else:
    print('Không tìm thấy yêu cầu với limit=40 trong log mạng.')

# Đóng trình duyệt
driver.quit()



Yêu cầu 1:
API URL: https://tiki.vn/api/personalish/v1/blocks/listings
Full URL: https://tiki.vn/api/personalish/v1/blocks/listings?limit=40&include=advertisement&aggregations=2&version=home-persionalized&trackity_id=06965c4d-cbf9-ea5f-2869-3463cc446426&category=8322&page=1&urlKey=nha-sach-tiki

Params:
limit: ['40']
include: ['advertisement']
aggregations: ['2']
version: ['home-persionalized']
trackity_id: ['06965c4d-cbf9-ea5f-2869-3463cc446426']
category: ['8322']
page: ['1']
urlKey: ['nha-sach-tiki']

Headers:
Accept: application/json, text/plain, */*
Referer: https://tiki.vn/nha-sach-tiki/c8322
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36
sec-ch-ua: "Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Windows"
x-guest-token: 3dSJWHLDtq6NRAnXPQaOChrkxjpi7GI8
